In [1]:
import requests
import os

In [12]:
# Function that allows us to obtain the dollar price, returning a dictionary with the rate and its date


def obtain_currency(api_key):
    url = "https://api.apilayer.com/exchangerates_data/latest"
    params = { "symbols": "ARS",  
        "base": "USD" }
    headers = {
        "apikey": api_key}
    response = requests.get(url, headers=headers, params=params)
    # Check the result of response, if some failure exist.
    if response.status_code == 200:
        data = response.json()
        cotizacion_dolar = data['rates']['ARS']
        print(f"Cotización actual del dólar: {cotizacion_dolar} ARS")
        registro = { 'date': data['date'] , 'price': data['rates']['ARS']}
        return registro
    else:
        print(f"Error al obtener la cotización: {response.status_code}, {response.text}")



In [7]:
obtain_currency(api_key)


Cotización actual del dólar: 975.742773 ARS


{'date': '2024-10-05', 'price': 975.742773}

In [13]:
import pandas as pd
import os

def append_to_data_price(path='./base_datos/data_price.csv'):
    #Save price of dollar in a variable.
    registro = obtain_currency(api_key)

    # Check if prices file (DF) exist, if not create one.
    if os.path.exists(path):
        data_prices = pd.read_csv(path)
    else:
        data_prices = pd.DataFrame(columns=['date', 'price'])

     # Convert the dictionary into a df, and add it to the prices df.
        
    nuevo_registro = pd.DataFrame([registro])
    data_prices = pd.concat([data_prices, nuevo_registro], ignore_index=True)

    # Save and up date prices df.      
    data_prices.to_csv(path, index=False)
    
    return data_prices

In [4]:
append_to_data_price(path='./base_datos/data_price.csv')

Cotización actual del dólar: 975.742773 ARS


,date,price
0,2024-10-05,975.742773
1,2024-10-06,975.742773


In [10]:
import os
from pathlib import Path

def transform_data (file1: str, file2: str): 
 
    base_dir = './base_datos/'
    base_path = Path(base_dir)
    file1_path = base_path / file1
    file2_path = base_path / file2
    
    # Check that if both files exist.
    
    if file1_path.exists() and file2_path.exists():
        # Read files.
        df1 = pd.read_excel(file1_path,engine='openpyxl')
        df2 = pd.read_csv(file2_path)


        # Get lasted record of prices dataset.
        df2['date'] = pd.to_datetime(df2['date'])
        latest_record = df2.sort_values(by='date').iloc[-1]

        # Transform last record to df.
        latest_record_df = pd.DataFrame([latest_record])  #
        
        # Concancate both df and calculate total amount in pesos.
        df_transform = pd.concat([df1, latest_record_df], ignore_index=True)
        df_transform['date'] = latest_record_df['date'].iloc[0]
        df_transform['price'] = round(df2['price'].iloc[0],2)
        df_transform['total_amount'] = df_transform['price'] * df_transform['Quantity']
                          
        # Save the changes in 'File 1' -->> stock_monetizado.xlsx
        
        df_transform.to_excel(file1_path, index=False)
        
        print(f"The files have been successfully concatenated and saved to {file1_path}")
    else:
        print("One or both files do not exist. Check names and paths.")
    
    

In [11]:
transform_data('stock_monetizado.xlsx','data_price.csv')

The files have been successfully concatenated and saved to base_datos\stock_monetizado.xlsx


In [9]:
df = pd.read_excel('./base_datos/stock_monetizado.xlsx',engine='openpyxl')
df

,PRODUCTO,Modelo,Descripcion,Proveedor,Unnamed: 4,Quantity,Price USD,date,price,total_amount
0,ID-00000001,BSX17,JUEGO DE TUBOS BAHCO 17P CON CRIQUE,ITURRIA S.A,BAHCO,1.0,1.322124,2024-10-06,975.74,975.74
1,ID-00000002,TM12,"TUBO ESTRIADO LARGO DE 1/2"" 12MM",ITURRIA S.A,BAHCO,2.0,0.720865,2024-10-06,975.74,1951.48
2,ID-00000003,TM13,"TUBO ESTRIADO LARGO DE 1/2"" 13MM",ITURRIA S.A,BAHCO,0.0,0.720865,2024-10-06,975.74,0.00
3,ID-00000004,TM14,"TUBO ESTRIADO LARGO DE 1/2"" 14 MM",ITURRIA S.A,BAHCO,0.0,0.720865,2024-10-06,975.74,0.00
4,ID-00000005,TM15,"TUBO ESTRIADO LARGO DE 1/2"" 15 MM",ITURRIA S.A,BAHCO,1.0,0.359955,2024-10-06,975.74,975.74
...,...,...,...,...,...,...,...,...,...,...
9836,ID-00009837,41400201204,CILINDRO COMPLETO STIHL FS 38/55,STIHL MOTOIMPLEMENTOS S.A,STIHL,1.0,70.110045,2024-10-06,975.74,975.74
9837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-06,975.74,NaN
9838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-06,975.74,NaN
9839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-06,975.74,NaN


In [ ]:
"C:\Users\Novakorp\Desktop\ITBA\data_aplications\etl_airflow\base_datos\stock_monetizado.xlsx"